# Limpeza e Processamento de Dados

Aqui extraímos os dados dos csv's, limpamos e processamos os dados, juntando o resultado em um único csv.

In [1]:
import pandas as pd
from utils import columns_to_drop, cid_translation, cat_dict

Aqui importamos a lista de colunas que decidimos não usar, seja por serem redundantes, não serem úteis ou por terem muitos dados faltantes. Além disso temos um dicionário de tradução dos números do CID, que especifica o tipo de doença, e separamos em sistemas do corpo humano para melhor visualização. Por último temos um dicionário para converter as colunas em tipos categóricos para diminuir o uso de memória e facilitar a análise.

In [16]:
# Converter o dicionario de traducao para que possa ser usado pela funcao .replace()
converted_cid_translation = {v: k for k, values in cid_translation.items() for v in values}

In [4]:
# Funcao generica para configurar categorias
def set_categories(df, col):
    df[col] = df[col].astype("category")
    df[col] = df[col].cat.set_categories(
        new_categories=cat_dict[col]["cats"],
        ordered=cat_dict[col]["ordered"],
    )
    return df

Carregamos o primeiro csv e limpamos os dados, removendo as colunas que decidimos não usar. 

In [5]:
df = pd.read_csv("data/POBR2013.csv")
df.drop(columns=columns_to_drop, inplace=True)

Agora, carregamos em sequência os demais csv's, limpando-os e juntando-os ao primeiro csv.

In [6]:
for i in range(14, 25):
    aux_df = pd.read_csv(f"data/POBR20{i}.csv")
    aux_df.drop(columns=columns_to_drop, inplace=True)
    df = pd.concat([df, aux_df])

In [7]:
# Aqui estamos substituindo os valores do CID pela traducao definida no dicionario
df["DIAG_DETH"] = df["DIAG_DETH"].replace(converted_cid_translation)

Agora, precisamos converter as colunas para tipos categóricos, para diminuir o uso de memória e facilitar a análise.

In [8]:
for col in df.columns:
    if col in cat_dict:
        df = set_categories(df, col)

Aqui vamos analisar se ainda temos dados faltantes nas colunas restantes.

In [12]:
for col in df.columns:
    print(f"{col}: {100 - df[col].isna().sum() / df.shape[0] * 100:.2f}%")

ANO_DIAGN: 100.00%
TRATAMENTO: 100.00%
DIAGNOSTIC: 100.00%
IDADE: 100.00%
SEXO: 100.00%
ESTADIAM: 93.15%
DIAG_DETH: 100.00%
DT_DIAG: 100.00%
DT_TRAT: 60.98%


A grande maioria das colunas não tem dados faltantes e no caso de `DT_TRAT` (data de início de tratamento) não há o que fazer, pois é possível que o paciente simplesmente não tenha recebido tratamento.

No caso de `ESTADIAM` (estadiamento), os dados já possuem um valor padrão para quando não há informação, que é `9`, então vamos apenas preencher os dados faltantes com esse valor.

In [17]:
df["ESTADIAM"].fillna(9, inplace=True)

Agora só resta converter o dataframe para um novo csv, que será usado na análise dos dados.

In [19]:
df.to_csv("data/POBR.csv", index=False)